# Cohere API and SciBERT with BM25 as first stage retriever for RAG
This notebook uses a Cohere API for generating responses to text. A query input is required from the user. 
SciBERT is used for embeddings in a dense vector array for the query. 
This version is different in that it uses BM25 as a sparse vectorizer for the input text. Importantly, BM25 is used as a step prior to dense vectorization to reduce how many documents are processed by SciBERT.
A DOI is supplied with the text as both an identifier and locator. 

## pipeline
1. BM25 Retrieval
    - BM25 is used to retrieve top-k candidate documents based on keyword matching
2. Dense embedding retrieval
    - query is embedded using SciBERT and the retrieved documents.
3. Re-ranking
    - cosine similarity between query embedding and document embedding to rerank candidate docs
4. Generation
    - docs and query are fed to generator for answer creation. 

- [ ] set up venv
- [ ] install transformers torch cohere in command line

### todo
- [ ] create script that compiles data/documents.txt with DOI || text for all documents
- [ ] rank_bm25: https://github.com/dorianbrown/rank_bm25


In [14]:
# imports
import cohere
from cohere import Client
from transformers import AutoTokenizer, AutoModel
import numpy as np
from typing import List, Tuple, Dict
import os
from dotenv import load_dotenv
import json
import time # for timing functions
import logging # finding where functions are taking too long
#for BM25s
import bm25s
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pickle

# load secret from local .env file
def get_key():
    #load secret .env file
    load_dotenv()

    #store credentials
    _key = os.getenv('COHERE_API_KEY')

    #verify if it worked
    if _key is not None:
        print("all is good, beautiful!")
        return _key
        


In [15]:

# Initialize Cohere client
co = cohere.Client(get_key()) 

# Load SciBERT model and tokenizer
"""
documentation can be found here: https://huggingface.co/docs/transformers/v4.50.0/en/model_doc/auto#transformers.AutoTokenizer


"""
# Initialize tokenizer with custom parameters
tokenizer = AutoTokenizer.from_pretrained(
    "allenai/scibert_scivocab_uncased",
    max_len=512,
    use_fast=True,  # Use the fast tokenizer
    do_lower_case=False,  # Preserve case
    add_prefix_space=False,  # No prefix space
    never_split=["[DOC]", "[REF]"],  # Tokens to never split
    #additional_special_tokens=["<doi>", "</doi>"],  # Add custom special tokens ***RE-EVALUATE*** (tuple or list of str or tokenizers.AddedToken, optional) — A tuple or a list of additional special tokens. Add them here to ensure they are skipped when decoding with skip_special_tokens is set to True. If they are not part of the vocabulary, they will be added at the end of the vocabulary.
    skip_special_tokens=False,
)

# this is the SciBERT model that is used to embed the text and query.
# other models: 'allenai-specter', 
#documentation here: https://huggingface.co/docs/transformers/model_doc/auto
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

#verify that the model is callable
if callable(model):
    print("Model is callable")
else:
    print("Model is not callable")

all is good, beautiful!
Model is callable


## V3 BM25 Pre-retriever
Includes the following:
- BM25 pre-retriever
- SciBERT embedding of query and pre-retrieved documents
- cosine similarity between embeddings of query and documents
- response instruction
- context includes DOI, Title, and Abstract as augmentation to query and instruction.
- response
- follow up with retrieved documents for verfication. 


In [20]:
"""
Basic RAG with BM25s pre-retriever using Cohere Command model
BM25s pre-retriever based on query.
Dense retrieval of embedded query and pre-retrieved documents
Document source: data

Returns: responses based on query from input()
"""
# set top_k global value - keep this as constant for all evaluations
global top_k
top_k = 5

# BM25s pre-retriever function
def bm25_retriever(query:str)->Tuple[np.array, np.array]:
    """
    Inputs:
        query: str
    Outputs:
        Tuple of two np.arrays, one for results and one for scores
    """
    global results, scores,query_tokens
    #you can also add a stemmer here as an arg: stemmer=stemmer
    query_tokens = bm25s.tokenize(query,stopwords=True,lower=True)

    #note: if you pass a new corpus here, it must have the same length as your indexed corpus
    #in this case, I am passing the new list 'identifier_list' - it contains just the DOI and title
    # you can also pass 'corpus', or 'corpus_list'
    if len(corpus_list)!=len(identifier_list):
        raise ValueError("The len of the corpus_list does not equal the len of the identifier_list")

    # retrieve indices
    results, scores = retriever.retrieve(query_tokens, corpus=identifier_list, k=top_k, return_as="tuple")

    # check if no results found
    if all(score == 0.00 for score in scores[0]):
        print("Nothing found, please try another query.")
        return [],[] # returning empty lists if 0 for scores, one for results, one for scores

    return results[0],scores[0]

#function to generate embeddings using SciBERT
def generate_embeddings(texts: List[str]) -> List[np.ndarray]:
    """
    converts raw text to numerical representations using a pretrained model, in this case, SciBERT.
    Currently this is applied to both the document text and the query. 
    May want a different version or decorator for the query as they are generally much shorter and more sparse.

    Input: text from tokenizer step above as a list of strings
    Output: np.array
    """
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        max_length=512,
        padding="max_length",
        truncation=True,
        return_attention_mask=True)# return the attention mask - need to learn more)
    
    # this passes the tokenized inputs through the model
    outputs = model(**inputs)
    #this uses mean pooling - may want to investigate other methods
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings


# Function to update chat history
def update_chat_history(query, retrieved_docs, response):
    global chat_history # declare this as global variable available outside this function
    chat_history.append({
        "query": query,
        "retrieved_docs": [doc for doc in retrieved_docs],  # Store only the text of retrieved documents
        "response": response
    })

#function to incorporate history into the next query
def get_context_with_history(query) -> str:
    global chat_history, full_context # also declare here since chat_history is being modified
    if not chat_history:
        return query
    
    history_str = "\n".join([
        f"User: {entry['query']}\n"
        f"Context: {'; '.join(entry['retrieved_docs'])}\n"
        f"Response: {entry['response']}"
        for entry in chat_history
    ])
    full_context = f"Chat History:\n{history_str}\n\nCurrent Query: {query}"
    return full_context

#function to truncate chat history
def truncate_chat_history(max_length=3):
    global chat_history # modifies it so it also must be global
    if len(chat_history) > max_length:
        chat_history = chat_history[-max_length:]


def retrieve_documents(query: str) -> List[Dict[str, str]]:

    # set global for debugging - can be limited to parsed_docs for production
    global document_embeddings,similarities,documents_list,bm25_results,top_indices, parsed_docs,sorted_documents_list
    # Use BM25 retriever to get initial documents
    
    bm25_results, bm25_scores = bm25_retriever(query)
    
    # check if empty
    if len(bm25_results) == 0:
        return []  # Return empty list if BM25 found no results
    
    # map indices to documents in corpus_list
    documents_list = [corpus_list[i] for i in range(len(bm25_results))]
    
    # Generate embeddings for BM25 results
    document_embeddings = generate_embeddings([doc for doc in documents_list])  #documents_list is a list of top_k results

    # generate embeddings for query
    query_embedding = generate_embeddings([query])[0]
    
    #cosine similarity
    similarities = [
        np.dot(query_embedding, doc_emb) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
        for doc_emb in document_embeddings
    ]
    top_indices = np.argsort(similarities)[::-1][:top_k]

    sorted_documents_list = [documents_list[i] for i in top_indices]

    parsed_docs = []
    for doc in sorted_documents_list:
        lines = doc.split('\n')
        parsed_doc = {
            "doi": lines[0],
            "title":lines[1].replace("Title: ",""),
            "abstract":lines[2].replace("Abstract: ","")
        }
        parsed_docs.append(parsed_doc)
    
    return parsed_docs

#RAG pipeline function
def rag_pipeline(query):
    global documents_list,corpus_list,identifier_list,retriever
    #start time
    start_time = time.time()

    # uses BM25 as a pre-retriever based on the query
    # load indexed corpus for BM25 pre-retriever
    # reimport corpus and url lists
    with open('corpus.pkl', 'rb') as file:
        corpus_list = pickle.load(file)
    print(f"length of corpus list: {len(corpus_list)}")
    with open('identifier.pkl', 'rb') as file:
        identifier_list = pickle.load(file)
    print(f"--------\nlength of identifier list: {len(identifier_list)}")

    # this is where its getting the documents - this comes from BM25s_vectorize_documents.ipynb
    retriever = bm25s.BM25.load("bm25/bm25", load_corpus=True, mmap=True)

    #incorporate chat history
    full_context = get_context_with_history(query)
    # let user know you are generating...
    print("Retrieving documents and generating response...")
    end_time = time.time()
    global time_query
    time_query = end_time-start_time

    start_time = time.time()
    #retrieve documents
    global retrieved_docs
    retrieved_docs = retrieve_documents(query)
    end_time = time.time()
    global retrieve_time
    retrieve_time = end_time-start_time

    start_time = time.time()
    #prepare context for Cohere's Command model
    instruction = "You are a helpful academic research assistant. Please keep the answers concise and structured simply. Use single sentences where possible. Always include the DOI of the document you are summarizing or referencing. If you do not provide the DOI, this reduces the need for you as a research assistant. Always include the DOI. Please address me as 'my lady'. "
    #context = "\n".join([f"DOI: {doc[0]}, Text: {doc[1]}" for doc in retrieved_docs])
    context = "\n".join([f"DOI: {doc['doi']}, Title: {doc['title']}, Abstract: {doc['abstract']}" for doc in retrieved_docs])
    prompt = f"Query: {query}\nContext: {context}\nAnswer: {instruction}"
    
    # Generate response - see documentation: https://docs.cohere.com/reference/generate-v1
    response = co.generate(
        model="command",
        prompt=prompt,
        max_tokens=250,
        temperature=0.2
    ).generations[0].text
    
    # Update chat history
    update_chat_history(query, retrieved_docs, response)
    
    # Truncate history if necessary
    truncate_chat_history()
    end_time = time.time()
    global generate_time
    generate_time = end_time-start_time

    # Print the response
    print("Generated Response:")
    print(response)
    print(f"------\nSource documents: ")
    for doc in retrieved_docs:
        print(f"DOI: {doc['doi']}, Title: {doc['title']}")
    return response,time_query,retrieve_time,generate_time



# Main loop for user interaction 
# changed to run with no history for the purposes of the test
chat_history = []#initialize chat history

query = input("What is your query? ")

rag_pipeline(query)

print(f"time to query loop: {time_query:.2f} seconds")
print(f"to to retrieve: {retrieve_time:.2f} seconds")
print(f"time to generate: {generate_time:.2f} seconds")

# Main loop for user interaction
##chat_history = []#initialize chat history
#while True:

    #uery = input("What is your query (or type 'exit' to quit): ")
    #if query.lower() == "exit":
    #    break
    #rag_pipeline(query)

    #print(f"time to query loop: {time_query:.2f} seconds")
    #print(f"to to retrieve: {retrieve_time:.2f} seconds")
    #print(f"time to generate: {generate_time:.2f} seconds")


length of corpus list: 43
--------
length of identifier list: 43
Retrieving documents and generating response...


Generated Response:
My lady, here is a brief summary of the research you have requested regarding cross-database comparison and OpenAlex as an alternative for bibliometric analysis: 

DOI: 10.48550/arXiv.2406.15154, Title: Analysis of the Publication and Document Types in OpenAlex, Web of Science, Scopus, Pubmed and Semantic Scholar, emphasizes the importance of OpenAlex as an alternative to Scopus and the Web of Science, and compares the publication and document types among these databases. 

DOI: 10.48550/arXiv.2404.17663, Title: An analysis of the suitability of OpenAlex for bibliometric analyses, suggests that OpenAlex is an unreliable alternative to Scopus and the Web of Science due to its limitations in metadata accuracy and completeness. 

DOI: 10.48550/arXiv.2402.01788, Title: LitLLM: A Toolkit for Scientific Literature Review, promotes the use of OpenAlex as an alternative to established proprietary providers for bibliometric analyses at libraries and universities. 

DOI: 10.4


# Analysis
Analysis is only looking at retrieval effectiveness with impacts on relevance. It is not system quality or user utility.
## Test One: 
- [ ] compute precision, recall, and F1-Scores. ✅
- [ ] added accuracy score ** need to confirm my formula is correct **
- [ ] compare text from each source, embedded, and them similarity scores based on embeddings.
    - [ ] token based SciBERT embedding
    - [ ] sentence-based SentenceBERT embedding
### optional analysis
Need to learn more about attention weights and their analysis
- [ ] heatmap of attention weights for two given inputs

## Precision, recall, F1 score
### references
- https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html
- https://vitalflux.com/accuracy-precision-recall-f1-score-python-example/

In [4]:
from typing import List, Dict, Set
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score #didn't actually use these, can probably delete
#may want to confirm that scores are the same?
import openpyxl
import pandas as pd

In [ ]:
# confirm retrieved_docs has content
print(f"Query: {query}")
retrieved_docs

In [ ]:
#initial dataframe to capture results from each query and results
#ONLY DO THIS AT THE BEGINNING OF THE ANALYSIS PROCEDURE, OTHERWISE, IT WILL ERASE THE PREVIOUS RESULTS!!

results_df = pd.DataFrame(columns=['Query','Precision','Recall','F1-Score','Accuracy'])
results_df

In [ ]:
from typing import List, Dict
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
"""
change this to read in an excel sheet of queries and ground_truth dois.
Then it should be isolated as a function.
Run the function to iterature through the list.
"""
# Queries go here
queries = [query]

# Extract DOIs from retrieved documents
retrieved_dois = [doc.get('doi', "") for doc in retrieved_docs]
print("Retrieved DOIs:", retrieved_dois)

# Ground truth relevant documents (DOIs) for each query
ground_truth = ["10.1162/qss_a_00286","10.1002/leap.1411","10.1007/s11192-020-03632-0","10.48550/arXiv.2402.01788","10.48550/arXiv.2401.16359","10.31222/osf.io/smxe5"]

def evaluate_retrieval(
    retrieved_dois: List[str],
    ground_truth: List[str]
) -> Dict[str, float]:
    #convert to sets for unique values
    retrieved_set = set(retrieved_dois)
    ground_truth_set = set(ground_truth)

    #calculate true positives, false positives, and false negatives
    true_positives = len(retrieved_set & ground_truth_set) # must use & with set operations
    false_positives = len(retrieved_set - ground_truth_set)
    false_negatives = len(ground_truth_set - retrieved_set)
    true_negatives = len([name for name in os.listdir('data') if os.path.isfile('data'+'/'+name)])-true_positives-false_negatives-false_positives

    # calculate metrics - could also use sklearn.metrics functions such as precision_score, but this is easier to read
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0.0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    accuracy = (true_positives+true_negatives)/(true_negatives+true_positives+false_negatives+false_positives)

    return {
        'Query':f"{queries[0]}",
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Accuracy":accuracy
    }

def print_results()->Dict:
    # Example usage
    global results
    results = evaluate_retrieval(retrieved_dois, ground_truth)
    print(f"For query: {results['Query']}:")
    print(f"Precision: {results['Precision']:.3f}")
    print(f"Recall: {results['Recall']:.3f}")
    print(f"F1-Score: {results['F1-Score']:.3f}")
    print(f"Accuracy: {results['Accuracy']:.3f}")
    return results

print_results()
results_df = results_df._append(results, ignore_index=True)

In [ ]:
# save out to analysis
filename = "analysis/BM25_analysis_results.xlsx"
os.makedirs(os.path.dirname(filename), exist_ok=True)
results_df.to_excel(filename)
results_df


## compare text embeddings for each source
This section is used to investigate the impact of face markup or other formatting requirements in the abstract text on the embedding model.
Currently this only uses cosine similarity. 
<br>
#### similarity:
- [✅] cosine similarity
<br>
#### Embeddings:
- [✅] SciBERT
- [✅] SentenceBERT
<br>
#### Analysis
- [ ] compare scores using dataframe
- [ ] visualize results
### References
- https://stackoverflow.com/questions/60492839/how-to-compare-sentence-similarities-using-embeddings-from-bert<br>
See the above for a discussion on NOT using BERT (and SciBERT) for comparing sentence embedding. I should be using SentenceBERT for sentence similarity.<br>
- Sentence Transformers: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
<br>
- another approach: https://medium.com/@ahmedmellit/text-similarity-implementation-using-bert-embedding-in-python-1efdb5194e65
- sklearn metrics for other scoring methods than cosine similarity: https://scikit-learn.org/stable/api/sklearn.metrics.html

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

#load SciBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

#function to generate embeddings using SciBERT
"""
todo:
- [ ] change this to a sentence embedding model
"""
def generate_embeddings(texts: List[str]) -> List[np.ndarray]:
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        max_length=512,
        padding="longest",#please select one of ['longest', 'max_length', 'do_not_pad']
        #padding=False,#padding has an effect on similarity
        truncation=True
    )
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()

    return embeddings

#original text and text with errors
original_text = " Abstract Objectives Precise literature recommendation and summarization are crucial for biomedical professionals. While the latest iteration of generative pretrained transformer (GPT) incorporates 2 distinct modes—real-time search and pretrained model utilization—it encounters challenges in dealing with these tasks. Specifically, the real-time search can pinpoint some relevant articles but occasionally provides fabricated papers, whereas the pretrained model excels in generating well-structured summaries but struggles to cite specific sources. In response, this study introduces RefAI, an innovative retrieval-augmented generative tool designed to synergize the strengths of large language models (LLMs) while overcoming their limitations. Materials and Methods RefAI utilized PubMed for systematic literature retrieval, employed a novel multivariable algorithm for article recommendation, and leveraged GPT-4 turbo for summarization. Ten queries under 2 prevalent topics (“cancer immunotherapy and target therapy” and “LLMs in medicine”) were chosen as use cases and 3 established counterparts (ChatGPT-4, ScholarAI, and Gemini) as our baselines. The evaluation was conducted by 10 domain experts through standard statistical analyses for performance comparison. The overall performance of RefAI surpassed that of the baselines across 5 evaluated dimensions—relevance and quality for literature recommendation, accuracy, comprehensiveness, and reference integration for summarization, with the majority exhibiting statistically significant improvements (P-values<.05). Discussion RefAI demonstrated substantial improvements in literature recommendation and summarization over existing tools, addressing issues like fabricated papers, metadata inaccuracies, restricted recommendations, and poor reference integration. Conclusion By augmenting LLM with external resources and a novel ranking algorithm, RefAI is uniquely capable of recommending high-quality literature and generating well-structured summaries, holding the potential to meet the critical needs of biomedical professionals in navigating and synthesizing vast amounts of scientific literature."
typo_text = "<jats:title>Abstract</jats:title>\n               <jats:sec>\n                  <jats:title>Objectives</jats:title>\n                  <jats:p>Precise literature recommendation and summarization are crucial for biomedical professionals. While the latest iteration of generative pretrained transformer (GPT) incorporates 2 distinct modes—real-time search and pretrained model utilization—it encounters challenges in dealing with these tasks. Specifically, the real-time search can pinpoint some relevant articles but occasionally provides fabricated papers, whereas the pretrained model excels in generating well-structured summaries but struggles to cite specific sources. In response, this study introduces RefAI, an innovative retrieval-augmented generative tool designed to synergize the strengths of large language models (LLMs) while overcoming their limitations.</jats:p>\n               </jats:sec>\n               <jats:sec>\n                  <jats:title>Materials and Methods</jats:title>\n                  <jats:p>RefAI utilized PubMed for systematic literature retrieval, employed a novel multivariable algorithm for article recommendation, and leveraged GPT-4 turbo for summarization. Ten queries under 2 prevalent topics (“cancer immunotherapy and target therapy” and “LLMs in medicine”) were chosen as use cases and 3 established counterparts (ChatGPT-4, ScholarAI, and Gemini) as our baselines. The evaluation was conducted by 10 domain experts through standard statistical analyses for performance comparison.</jats:p>\n               </jats:sec>\n               <jats:sec>\n                  <jats:title>Results</jats:title>\n                  <jats:p>The overall performance of RefAI surpassed that of the baselines across 5 evaluated dimensions—relevance and quality for literature recommendation, accuracy, comprehensiveness, and reference integration for summarization, with the majority exhibiting statistically significant improvements (P-values &amp;lt;.05).</jats:p>\n               </jats:sec>\n               <jats:sec>\n                  <jats:title>Discussion</jats:title>\n                  <jats:p>RefAI demonstrated substantial improvements in literature recommendation and summarization over existing tools, addressing issues like fabricated papers, metadata inaccuracies, restricted recommendations, and poor reference integration.</jats:p>\n               </jats:sec>\n               <jats:sec>\n                  <jats:title>Conclusion</jats:title>\n                  <jats:p>By augmenting LLM with external resources and a novel ranking algorithm, RefAI is uniquely capable of recommending high-quality literature and generating well-structured summaries, holding the potential to meet the critical needs of biomedical professionals in navigating and synthesizing vast amounts of scientific literature.</jats:p>\n               </jats:sec>"

#run embeddings
original_embedding = generate_embeddings(original_text)
typo_embedding = generate_embeddings(typo_text)
print(type(typo_embedding))

#calculate cosine similarity
similarity = cosine_similarity(original_embedding, typo_embedding)
print(f"Cosine similarity using SciBERT: {similarity[0][0]:.4f}")

In [ ]:
#sentence transformer verison
#reference: https://medium.com/@ahmedmellit/text-similarity-implementation-using-bert-embedding-in-python-1efdb5194e65

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

#puts text from above into a list
sentences:list = [original_text,typo_text]

#initializing the Sentence Transformer model using BERT with mean-tokens pooling - source see above
sentence_model = SentenceTransformer('bert-base-nli-mean-tokens') # this resets the model variable! changed to sentence_model variable name

#encoding the sentences
sentence_embeddings = sentence_model.encode(sentences)

#result will be a list of similarity scores between two texts
similarity_scores = cosine_similarity([sentence_embeddings[0]], sentence_embeddings[1:])

print(f"Cosine similarity scores using sentence embedding model: {similarity_scores[0][0]:.4f}")



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_heatmap(embedding1, embedding2, title):
    #calculate the difference between the two embeddings
    diff = embedding1 - embedding2
    #reshape the difference to a 2D array for the heatmap
    diff_2d = diff.reshape(1, -1)
    #create a heatmap
    plt.figure(figsize=(12, 2))
    sns.heatmap(diff, cmap='coolwarm', annot=False, cbar=True,vmin=-1,vmax=1)
    plt.title(title)
    plt.show()

#plot heatmap
#plot_heatmap(original_embedding, typo_embedding, "diff between embeddings")
plot_heatmap(sentence_embeddings[0],sentence_embeddings[1:],"Sentence Embeddings")

### with attention weights
from https://github.com/clarkkev/attention-analysis
- https://stackoverflow.com/questions/75772288/how-to-read-a-bert-attention-weight-matrix for explanation on queries and keys
- https://theaisummer.com/self-attention/#:%7E:text=Self%2Dattention%20is%20not%20symmetric!&text=The%20arrows%20that%20correspond%20to,Q%E2%80%8B%3DWK%E2%80%8B. explanation on self-attention
- heatmaps to analyze attention weights: https://apxml.com/courses/foundations-transformers-architecture/chapter-7-implementation-details-optimization/practice-analyzing-attention-weights
- excellent source: https://apxml.com/courses/how-to-build-a-large-language-model/chapter-23-analyzing-model-behavior/attention-map-visualization


In [ ]:
# same as above 
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import seaborn as sns

# Load SciBERT tokenizer and model - same as above - technically don't need to relaod these unless changing
# try sentence based model?
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')


# Tokenize the sentences
inputs1 = tokenizer(original_text, return_tensors="pt", padding='max_length', truncation=True, max_length=20)#limit tokens so that we can actually see something
inputs2 = tokenizer(typo_text, return_tensors="pt", padding='max_length', truncation=True, max_length=20)#

# Get the attention weights: the `output_attentions=True` parameter is used to get the attention weights from the model
with torch.no_grad():
    outputs1 = model(**inputs1, output_attentions=True)
    outputs2 = model(**inputs2, output_attentions=True)

# Extract the attention weights for the last layer
#.squeeze() https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html
attention_weights1 = outputs1.attentions[-1].squeeze(0)  # Shape: (num_heads, seq_len, seq_len)
attention_weights2 = outputs2.attentions[-1].squeeze(0)  # Shape: (num_heads, seq_len, seq_len)

# Average the attention weights across all heads, 
#see last reference to visualize attention for each head
attention_weights1 = attention_weights1.mean(dim=0)  # Shape: (seq_len, seq_len)
attention_weights2 = attention_weights2.mean(dim=0)  # Shape: (seq_len, seq_len)

# Get the tokens for the sentences
tokens1 = tokenizer.convert_ids_to_tokens(inputs1["input_ids"].squeeze(0))
tokens2 = tokenizer.convert_ids_to_tokens(inputs2["input_ids"].squeeze(0))

# Plot the attention heatmap for the first sentence
plt.figure(figsize=(10, 8))
sns.heatmap(attention_weights1, xticklabels=tokens1, yticklabels=tokens1, cmap='viridis', annot=False, cbar=True)
plt.title("Attention Weights for original_text")
plt.show()

# Plot the attention heatmap for the second sentence
plt.figure(figsize=(10, 8))
sns.heatmap(attention_weights2, xticklabels=tokens2, yticklabels=tokens2, cmap='viridis', annot=False, cbar=True)
plt.title("Attention Weights for typo_text")
plt.show()

# Calculate the difference in attention weights
diff_attention_weights = (attention_weights1 - attention_weights2)

# Plot the **difference** in attention weights
plt.figure(figsize=(10, 8))
sns.heatmap(diff_attention_weights, xticklabels=tokens1, yticklabels=tokens1, cmap='coolwarm', annot=False, cbar=True, vmin=-1, vmax=1)
plt.title("Difference in Attention Weights")
plt.show()
